# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will convert the json encoded request body into the image format. There are two main functions in the API: init() and run(). The init() function loads the model and returns a scoring function. The run() function process the images and uses the first function to score them.

    Note: Always make sure you don't have any lingering notebooks running (Shutdown previous notebooks). Otherwise it may cause GPU memory issue.

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
from dotenv import set_key, find_dotenv
import logging
from testing_utilities import get_auth

In [ ]:
# Import Deep learning libraries
from fastai.vision import open_image, load_learner
import torchvision.models as models
import torch
import fastai

In [ ]:
print("PyTorch: ", torch.__version__)
print("FastAI: ", fastai.__version__)

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
%%writefile driver.py
# Driver for Model  

import timeit as t
import logging
import os
from azureml.core.model import Model
from fastai.vision import open_image, load_learner
from azureml.contrib.services.aml_request import rawhttp
from azureml.core.model import Model
from toolz import compose


def _create_scoring_func():
    """ Initialize ResNet 18 Model
    """
    logger = logging.getLogger("model_driver")
    start = t.default_timer()
    model_name = "resnet_model"
    model_path = Model.get_model_path(model_name=model_name)
    model_dir_path, model_filename = os.path.split(model_path)
    model = load_learner(path=model_dir_path, fname=model_filename)
    end = t.default_timer()

    loadTimeMsg = "Model loading time: {0} ms".format(round((end - start) * 1000, 2))
    logger.info(loadTimeMsg)

    def call_model(img_ref):
        img = open_image(img_ref)
        pred_class, pred_idx, outputs = model.predict(img)
        decoded_predictions = [str(pred_class), outputs[pred_idx].item()]
        return [decoded_predictions]

    return call_model


def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = _create_scoring_func()

    def process_and_score(images_dict):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        logger.info("Scoring {} images".format(len(images_dict)))
        preds = {key: scoring_func(img_ref) for key, img_ref in images_dict.items()}
        end = t.default_timer()

        logger.info("Predictions: {0}".format(preds))
        logger.info("Predictions took {0} ms".format(round((end - start) * 1000, 2)))
        return (preds, "Computed in {0} ms".format(round((end - start) * 1000, 2)))

    return process_and_score


def init():
    """ Initialise the model and scoring function
    """
    global process_and_score
    process_and_score = get_model_api()


@rawhttp
def run(request):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    return process_and_score(request.files)

## Test the driver

In [ ]:
# We test the driver by passing data.
logging.basicConfig(level=logging.DEBUG)
%run driver.py

In [ ]:
# Let's load the workspace.
ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

In [ ]:
model_path = Model.get_model_path("resnet_model", _workspace=ws)

In [ ]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [ ]:
# Always make sure you don't have any lingering notebooks running. Otherwise it may cause GPU memory issue.
process_and_score = get_model_api()

In [ ]:
resp = process_and_score({"lynx": open("220px-Lynx_lynx_poing.jpg", "rb")})
resp

In [ ]:
# Clear GPU memory 
torch.cuda.empty_cache()# Next, we will [build a docker image with this modle driver and other supporting files](03_BuildImage.ipynb).